# Translation Evaluator (*^▽^*)

In [5]:
# 🔧 文件选择参数设置
print("=== 文件选择参数设置 ===")

# ========== 在这里修改参数 ==========
# 只需要指定文件名的关键词，系统会自动匹配
article_filename = "the-role"           # 文章文件名关键词
translation_filename = "the-role"   # 翻译文件名关键词（可选，留空则自动匹配）
# ===================================

print(f"🎯 目标文章关键词: {article_filename}")
print(f"📝 目标翻译关键词: {translation_filename}")
print("\n参数设置完成！")

=== 文件选择参数设置 ===
🎯 目标文章关键词: the-role
📝 目标翻译关键词: the-role

参数设置完成！


In [6]:
# 导入模块和初始化
from modules.config_manager import load_config, init_openai_client
from modules.translation_evaluator import (
    get_available_articles, get_available_translations, display_available_files,
    extract_article_content, read_user_translation, evaluate_translation_with_ai,
    generate_evaluation_report, display_evaluation_preview, find_matching_files
)
from modules.file_manager import extract_title_from_article, ensure_directory


print("=== 翻译评估系统初始化 ===")

# 加载配置
config = load_config()
client, model = init_openai_client(config)

print(f"✅ 配置加载成功！使用模型: {model}")
print("✅ 系统初始化完成！")

=== 翻译评估系统初始化 ===
✅ 配置加载成功！使用模型: gpt-4o
✅ 系统初始化完成！


In [7]:
# 📂 文件选择和验证
print("=== 文件选择和验证 ===")

# 获取可用文件
articles = get_available_articles()
translations = get_available_translations()

# 显示可用文件
display_available_files(articles, translations)

if not articles:
    print("❌ 没有找到文章文件！")
    print("请先运行文章生成系统，或在 articles/ 文件夹中添加 .md 文件")
elif not translations:
    print("❌ 没有找到翻译文件！")
    print("请在 translations/ 文件夹中添加 .txt 格式的翻译文件")
else:
    try:
        # 查找匹配的文件
        article_path, translation_path = find_matching_files(
            article_filename, translation_filename
        )
        
        # 提取文章标题
        article_title = extract_title_from_article(article_path)
        
        print(f"\n✅ 文件匹配成功！")
        print(f"📖 选中的文章: {article_path.name}")
        print(f"📝 选中的翻译: {translation_path.name}")
        print(f"🏷️  文章标题: {article_title}")
        
        print("\n可以继续运行下一步开始评估。")
        
    except FileNotFoundError as e:
        print(f"❌ 文件匹配失败: {e}")
        print("请检查文件名设置，或手动修改参数")

=== 文件选择和验证 ===
可用的文章文件：
  1. academic_words-Collaboration-in-Scriptwriting.md
  2. academic_words-The-Art-of-Scriptwriting.md
  3. academic_words-The-Impact-of-Technology-on-Scriptwriting.md
  4. list1-Navigating-Financial-Markets-20250819_165623.md
  5. list1-The-Role-of-Governance-in-Business-20250819_165623.md
  6. list1-Understanding-Real-Estate-Transactions-20250819_165623.md

可用的翻译文件：
  1. navigating-financial-markets.txt
  2. the-role-of-governance-in-business.txt

✅ 文件匹配成功！
📖 选中的文章: list1-The-Role-of-Governance-in-Business-20250819_165623.md
📝 选中的翻译: the-role-of-governance-in-business.txt
🏷️  文章标题: The Role of Governance in Business

可以继续运行下一步开始评估。


In [8]:
# 🤖 执行AI翻译评估
print("=== 开始AI翻译评估 ===")

# 确保评估结果目录存在（不删除现有报告）
ensure_directory('translation_results')

if 'article_path' in locals() and 'translation_path' in locals():
    try:
        print(f"📖 正在分析文章: {article_title}")
        print(f"📝 正在读取翻译: {translation_path.name}")
        
        # 提取文章内容
        english_sentences = extract_article_content(article_path)
        print(f"📊 文章包含 {len(english_sentences)} 句")
        
        # 读取用户翻译
        user_translations = read_user_translation(translation_path)
        print(f"📊 翻译包含 {len(user_translations)} 句")
        
        if len(english_sentences) != len(user_translations):
            print("⚠️  警告: 原文句数与翻译句数不匹配，将使用较短的长度进行评估")
        
        print("\n⏳ 正在调用AI进行详细评估，请稍候...")
        
        # 执行AI评估
        evaluation_result = evaluate_translation_with_ai(
            client=client,
            model=model,
            english_sentences=english_sentences,
            user_translations=user_translations
        )
        
        if evaluation_result:
            print("✅ AI评估完成！正在生成报告...")
            
            # 生成评估报告
            report_path, report_content = generate_evaluation_report(
                evaluation_result=evaluation_result,
                article_title=article_title,
                translation_filename=translation_path.name,
                model=model
            )
            
            print(f"\n🎉 评估报告生成完成！")
            print(f"📄 报告文件: {report_path.name}")
            print(f"📁 保存路径: {report_path}")
            
            # 显示报告预览
            display_evaluation_preview(report_content)
            
            print(f"\n📂 完整报告保存在 translation_results/ 文件夹中")
            print("✨ 您现在可以查看详细的评估报告和改进建议")
            
        else:
            print("❌ AI评估失败，请检查网络连接和API配置")
            
    except Exception as e:
        print(f"❌ 评估过程中出错: {e}")
        
else:
    print("❌ 请先完成文件选择和验证步骤")

=== 开始AI翻译评估 ===
已确保目录存在: translation_results
📖 正在分析文章: The Role of Governance in Business
📝 正在读取翻译: the-role-of-governance-in-business.txt
📊 文章包含 7 句
📊 翻译包含 7 句

⏳ 正在调用AI进行详细评估，请稍候...
原文句数: 7
翻译句数: 7
✅ AI评估完成！正在生成报告...

🎉 评估报告生成完成！
📄 报告文件: AI_translation_evaluation_20250820_162900.md
📁 保存路径: translation_results\AI_translation_evaluation_20250820_162900.md

=== 报告预览 ===
# AI翻译评估报告

## 基本信息
- **原文章标题**: The Role of Governance in Business
- **翻译文件**: the-role-of-governance-in-business.txt
- **评估时间**: 2025-08-20 16:29:00
- **评估模型**: gpt-4o

## 详细评估结果

## **SENTENCE 1**

**原文**: Effective **governance** is vital for any business  
**用户翻译**: 高效的政府部门对任何生意都很重要  
**标准翻译**: 有效的**治理**对任何企业都至关重要  
***流畅翻译***: 企业的有效**治理**对其成功至关重要  
**评分**: 4  
**评价**: 用户将“governance”误译为“政府部门”，而实际上应为“治理”或“管理”。***词汇***选择不当导致翻译意思偏离原文。建议更正为“治理”以准确表达原文意思。

## **SENTENCE 2**

**原文**: It involves...

📂 完整报告保存在 translation_results/ 文件夹中
✨ 您现在可以查看详细的评估报告和改进建议
